In [8]:
class ExtractMeasurements:
    def __init__(self,c8y,source):
        '''
        ::PARAMETERS::
        c8y -> instância 'CumulocityApi' da lin c8y-api
        source -> id do device dentro da instância do Cumulocity IoT
        ::DESCRIPTION::
        inicia os dois objetos para utilizarmos nas demais funções
        as variaveis de timedelta são iniciadas aqui pois elas irão controlar o periodo da requisição na API

        '''
        
        from datetime import timedelta
        import pandas as pd

        #instância necessárias para realizar as operaçoes na API
        self.c8y = c8y
        self.source = source

        self.delta_twohours = timedelta(hours=2) # 2 hours older
        self.delta_onehour = timedelta(hours=1) # 1 hour older


    #classe para extrair de devices novos
    def extract_new_devices(self) -> list:
        '''
        ::PARAMETERS::
            Não possui.
        ::DESCRIPTION::
            há diferentes devices com formas de aquisição de dados diferentes pois a forma de subir para a nuvem do Cumulocity IoT foi diferente.
            Esse objeto contempla a extração de dados de devices que foram alocados no Cumulocity via conector do eWon. Tal forma de subir os dados é tendência por sua simplicidade e alta capacidade de controle de time interval.
        ::RETURN::
            retorna uma lista com dois dataframes.
            o df[0] é o dataframe com dados de UMA hora atrás
            o df[1] é o dataframe com dados de DUAS hora atrás
        '''
        def tratamento(_hour_measu):
            '''
            ::PARAMETER:: 
                _hour_measu -> generator gerado por CumulocityApi.measurements.select()
            :::DESCRIPTION::
                formata os dados requisitados num dataframe inserindo para cada dado de nome novo uma nova coluna no dataframe
                Alguns dados podem ter o intervalo de aquisição diferente, para isso, inserimos a var 'comp_min' para equilibrar os arrays e inserir num dataframe de linhas iguais.
            ::RETURN:::
                retorna o dataframe dinamicamente formatado
            '''
            df = {}
            for i,v in enumerate(_hour_measu):
                fragments = v.fragments # caracteristicas do dados puxado. Daí tiramos as propriedes que queremos
                print(fragments)
                col_name = list(fragments.keys())[0] # nome da nossa coluna no df
                print(col_name)
                aux = list(fragments[col_name].keys())[0] # parametro utilizado para entrar no aninhamento
                print(aux)
                value = [list(fragments[col_name][aux].values())[1]] #valor do parametro atual -> inserimos ele numa lista para poder appendar no script abaixo
                print(value)

                # no primeiro loop, precisamos definir as colunas do dataframe, nos próximos basta appendarmos.
                try:
                    df[col_name].append(value[0])
                except KeyError as error:
                    df[col_name] = value

            comp_min = min(map(len,df.values())) # alguns parametros possuem time interval de envio de dados diferentes. Isso faz com que seja necessario equilibrarmos ele
                #limitando as linhas do dataframe
            for k,v in df.items():
                    df[k] = v[:comp_min]
                
            df = pd.DataFrame(df)
            return df
        
        #measurements de uma hora atrás
        one_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_onehour)
        #measurements de duas horas atras
        two_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_twohours, min_age=self.delta_onehour)
        
        # retorna uma lista com os dfs de uma hora atrás e duas horas atrás da mesma source
        return [tratamento(one_hour_measu),tratamento(two_hour_measu)]


    def extract_old_devices(self):
        def tratamento(_hour_measu):
            df = {}
            for i,v in enumerate(_hour_measu):
                fragments = v.fragments.values() # taking the actual parameter
                for f in fragments:
                    col_name = list(f.keys())[0] # setting 
                    value = [list(f[col_name].values())[1]]
                    
                    try:
                        df[col_name].append(value[0])
                    except KeyError as error:
                        df[col_name] = value

            comp_min = min(map(len,df.values()))
            for k,v in df.items():
                df[k] = v[:comp_min]
            df = pd.DataFrame(df)

            return df

        #measurements de uma hora atrás
        one_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_onehour)
        #measurements de duas horas atras
        two_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_twohours, min_age=self.delta_onehour) 

        return [tratamento(one_hour_measu),tratamento(two_hour_measu)]        
    
        
        

# TESTE DA CLASSE

In [19]:
# libs necessárias
from c8y_api import CumulocityApi
from datetime import timedelta
import pandas as pd 

import json
import os

config_path = os.path.join(os.getcwd(), '..','..','..','','configs','config.json')
with open(config_path) as config_file:
    login = json.load(config_file)

# instâncias gloabais
c8y = CumulocityApi(
    base_url= login['url'],
    tenant_id= login['tenant'],
    username= login['username'],
    password= login['password'],
)

#devices de teste
# 621151045972 -> unilever || 1069740447 -> serbom || 611284292311 -> garoto cag 09 || 481304022994 -> garoto cag 10 || 891204422984 -> Fini Secomea || 581244909717 -> SODEBO
sources = ['621151045972', '1069740447', '611284292311','481304022994','891204422984', '581244909717']


In [20]:
extract = ExtractMeasurements(c8y=c8y,source=sources[3])

In [23]:
extract.extract_new_devices()[0]

,S01_AC_02_CAG10-200,S02_AC_02_CAG10-200,S03_AC_02_CAG10-200,SP_CAG10-200,DP_CAG10-200,SP_CAG10-100,DT_CAG10-200,OP_CAG10-200,ST_CAG10-200,OT_CAG10-200,...,S03_AC_01_CAG10-100,S04_AC_01_CAG10-100,S01_VEL_02_CAG10-200,S02_VEL_02_CAG10-200,S03_VEL_02_CAG10-200,S04_VEL_02_CAG10-200,S01_VEL_01_CAG10-100,S02_VEL_01_CAG10-100,S03_VEL_01_CAG10-100,S04_VEL_01_CAG10-100
0,5508.639,8696.482,1085.264,3.575967,13.52000,3.284690,72.26025,16.80835,3.638725,48.94967,...,656.1763,482.9955,1.405251,4.184626,0.459281,0.534338,3.402910,1.748987,0.907778,0.842607
1,5657.777,8719.098,1096.929,3.582229,13.51392,3.357024,72.18077,16.85156,3.560402,48.94967,...,617.5363,428.3842,1.411509,6.332833,0.584019,0.770911,3.482302,1.774470,0.629804,0.725396
2,5308.669,8721.613,1120.997,3.577262,13.55286,3.330250,72.09243,16.89294,3.480026,48.65523,...,659.8561,448.8789,1.319715,8.378495,0.932205,1.164104,3.496381,1.713655,1.089948,0.953938
3,5658.306,8843.858,1071.397,3.562580,13.55590,3.287498,71.88630,16.93857,3.405827,48.13113,...,669.9769,489.2466,1.494628,6.527647,0.667845,0.864776,3.476523,1.819954,1.136710,1.183027
4,5699.139,9182.220,1134.701,3.554375,13.61310,3.249711,71.69789,17.00916,3.335754,48.07519,...,711.6836,461.1174,1.376649,6.684771,0.655811,0.850568,3.362514,1.777141,0.808819,0.864928
5,5476.896,9574.810,1115.639,3.533431,13.60884,3.357887,71.63898,16.97448,3.251247,48.42557,...,645.5054,444.2863,1.418008,8.010087,0.671296,0.887839,3.543426,1.756836,0.677115,0.756118
6,5701.215,9370.977,1153.925,3.538829,13.64840,3.315999,71.86570,16.97752,3.259487,48.94083,...,640.8891,554.9770,1.476902,7.425537,0.749004,0.992260,3.527920,1.771860,1.984519,1.853255
7,6183.358,9347.155,1104.961,3.531056,13.69951,3.289225,71.89809,17.04506,3.240944,48.24302,...,697.3537,486.7930,1.406889,5.747694,0.873352,1.183845,3.336782,1.771226,0.793629,1.066819
8,5964.543,9646.076,1212.371,3.513998,13.69464,3.405606,71.93048,16.91423,3.160561,49.00267,...,621.8105,453.4907,1.330559,6.382072,0.787839,1.111746,3.753466,1.813673,0.660079,0.686642
9,5655.986,8877.666,1172.932,3.534942,13.65996,3.373650,72.38097,16.94222,3.255371,50.24519,...,654.8749,424.3385,1.377335,6.626056,0.669640,0.832079,3.468865,1.753614,0.658017,0.719430
